# Team 10 Biopharma Case

### Get FX data for the Biopharma case and fit multivariate normal distribution for subsequent sampling and simulation.

In [1]:
!pip install forex_python

from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta

import numpy as np
from scipy.stats import multivariate_normal


In [2]:
def get_monthly_exchange_rates(start_year, end_year, base_currency, target_currencies):
    c = CurrencyRates()
    rates = {currency: [] for currency in target_currencies}

    # Generate a list of first day of each month in the specified range
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    current_date = start_date

    while current_date <= end_date:
        for currency in target_currencies:
            try:
                # Get exchange rate for the first day of the month
                rate = c.get_rate(base_currency, currency, current_date)
                rates[currency].append((current_date.strftime("%Y-%m"), rate))
            except Exception as e:
                print(f"Error fetching rate for {currency} on {current_date.strftime('%Y-%m')}: {e}")
                rates[currency].append((current_date.strftime("%Y-%m"), None))

        # Move to the first day of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return rates

In [3]:
# Specify the currencies
base_currency = "USD"
target_currencies = ["BRL", "EUR", "INR", "JPY", "MXN", "USD"]

# Fetch exchange rates from Jan 2019 to Dec 2023
exchange_rates = get_monthly_exchange_rates(2019, 2023, base_currency, target_currencies)

# Print the rates
for currency, rates in exchange_rates.items():
    print(f"Exchange rates for {currency} against {base_currency}:")
    for date, rate in rates:
        print(f"{date}: {rate}")
    print("\n")



Exchange rates for BRL against USD:
2019-01: 3.8812227074235808
2019-02: 3.6709964257693315
2019-03: 3.7808134938065536
2019-04: 3.8771804912780348
2019-05: 3.9267249064004286
2019-06: 3.9872657160792757
2019-07: 3.8272094457661465
2019-08: 3.8255866630424937
2019-09: 4.157212758245742
2019-10: 4.164433841071756
2019-11: 3.989316814794865
2019-12: 4.230468038608632
2020-01: 4.0196724230016025
2020-02: 4.266829533116178
2020-03: 4.485014120433634
2020-04: 5.2440563277249455
2020-05: 5.384792203015815
2020-06: 5.3324937027707815
2020-07: 5.442857142857142
2020-08: 5.167032410533423
2020-09: 5.433052473512972
2020-10: 5.6008339006126615
2020-11: 5.779363993845102
2020-12: 5.311915106951871
2021-01: 5.193953223046206
2021-02: 5.442320423700762
2021-03: 5.579025968638513
2021-04: 5.631619274646687
2021-05: 5.346548584671412
2021-06: 5.202126789366054
2021-07: 4.960871760350051
2021-08: 5.1060465898578755
2021-09: 5.152407548447152
2021-10: 5.415172413793104
2021-11: 5.650889618241493
2021-1

In [10]:
# Convert the data into a NumPy array for analysis
# Note: This example assumes all currencies have the same number of rates and ignores missing values

# Extract rates for each currency, ensuring they're in the same order for each month
rates_list = [rates for rates in exchange_rates.values() if all(rate is not None for _, rate in rates)]
currency_data = np.array([[rate for _, rate in currency_rates] for currency_rates in rates_list])

# Transpose to get rows as observations (dates) and columns as variables (currencies)
currency_data = currency_data.T

# Ensure no NaN values; this method requires complete data
if np.isnan(currency_data).any():
    print("Data contains NaN values. Please handle missing data before proceeding.")
else:
    # Estimate the mean and covariance
    mean_vector = np.mean(currency_data, axis=0)
    covariance_matrix = np.cov(currency_data, rowvar=False)

    # Fit the multivariate normal distribution
    mvn_distribution = multivariate_normal(mean=mean_vector, cov=covariance_matrix, allow_singular=True)

    print("Mean Vector:\n", mean_vector)
    print("Covariance Matrix:\n", covariance_matrix)
    # The distribution is now defined and can be used for further analysis


Mean Vector:
 [  4.91679403   0.89791993  75.7301565  119.26608249  19.81262121
   1.        ]
Covariance Matrix:
 [[ 3.42594387e-01 -2.13708474e-03  1.22952065e+00  1.27836101e+00
   3.46111090e-01  0.00000000e+00]
 [-2.13708474e-03  2.17002114e-03  1.24913080e-01  5.21137097e-01
  -1.73246912e-02  0.00000000e+00]
 [ 1.22952065e+00  1.24913080e-01  2.02929685e+01  6.06245515e+01
  -2.53203458e+00  0.00000000e+00]
 [ 1.27836101e+00  5.21137097e-01  6.06245515e+01  2.26189966e+02
  -1.36106797e+01  0.00000000e+00]
 [ 3.46111090e-01 -1.73246912e-02 -2.53203458e+00 -1.36106797e+01
   2.22048686e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [11]:
sample = mvn_distribution.rvs(size=1, random_state = 42)

print("Sampled draw:", sample)

Sampled draw: [  5.33903349   0.86624389  73.11981139 111.97694712  20.64765002
   1.        ]


# Biopharma Case Simulation Section

## Install and import packages

In [5]:
# Install and import packages
!pip install gurobipy
!pip install tabulate

import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum
from tabulate import tabulate
import datetime as dt
_empty_series = pd.Series(dtype=float)

## Raw data

In [7]:
selected_yr = 2023
base_yr = 2019

demand = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'd_h': [  7, 15,  5,  7,  3, 18],
    'd_r': [  7, 12,  3,  8,  3, 17],
})
demand.set_index('from', inplace=True)

caps = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'cap': [18, 45, 18, 10, 30, 22],
})
caps.set_index('plant', inplace=True)

pcosts = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'fc_p': [20, 45, 14, 13, 30, 23],
    'fc_h': [ 5, 13,  3,  4,  6,  5],
    'fc_r': [ 5, 13,  3,  4,  6,  5],
    'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
    'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
    'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
    'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
})
pcosts.set_index('plant', inplace=True)

tcosts = pd.DataFrame({
    'from': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'LatinAmerica': [ 0.20, 0.45, 0.50, 0.50, 0.40, 0.45],
    'Europe':       [ 0.45, 0.20, 0.35, 0.40, 0.30, 0.30],
    'AsiaWoJapan':  [ 0.50, 0.35, 0.20, 0.30, 0.50, 0.45],
    'Japan':        [ 0.50, 0.40, 0.30, 0.10, 0.45, 0.45],
    'Mexico':       [ 0.40, 0.30, 0.50, 0.45, 0.20, 0.25],
    'U.S.':           [ 0.45, 0.30, 0.45, 0.45, 0.25, 0.20],
})
tcosts.set_index('from', inplace=True)

duties = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'duty': [ 0.30, 0.03, 0.27, 0.06, 0.35, 0.04],
})
duties.set_index('from', inplace=True)

# Your provided exchange_rate_data
exrate = {
    '2018': [3.88, 4.33, 69.63, 109.91, 19.64, 1],
    '2019': [4.33, 0.92, 71.48, 109.82, 18.65, 1],
    '2020': [5.19, 0.82, 73.66, 103.24, 19.90, 1],
    '2021': [5.26, 0.88, 74.28, 115.59, 20.62, 1],
    '2022': [5.29, 0.93, 82.75, 131.12, 19.48, 1],
    '2023': [4.85, 0.91, 83.04, 140.99, 16.96, 1],
}
exrate = pd.DataFrame(exrate , index=['BRL', 'EUR', 'INR', 'JPY', 'MXN', 'USD'])

### Functions to calculate cost, unmet demand, and excess capacity

In [8]:
# identify number of supply and demand location for iterations
n_ctry = range(demand.shape[0])
n_lines = range(demand.shape[1]+1)

# Objective function to calculate cost
def calc_total_cost(dec_plant, dec_h, dec_r, base_yr, exrate_fun, tariff=0):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    # Adjust the cost using exchange rate of give year
    reindx = exrate.loc[:, f'{base_yr}'] / exrate_fun
    pcosts_rev = pcosts.values * reindx.values.reshape(-1,1)
    pcosts_rev = pd.DataFrame(pcosts_rev, columns=pcosts.columns[0:], index=pcosts.index)
    # return pcosts_rev
    # pcosts_rev = adj_pcosts_exrate(2019, 2023)

    duties_mat = np.zeros(len(duties)) + (1 + duties['duty'].values)[:, np.newaxis]
    np.fill_diagonal(duties_mat, 1)
    duties_mat = pd.DataFrame(duties_mat.T, index=pcosts_rev.index, columns=duties.index)
    duties_mat.loc['Germany', 'U.S.'] += tariff
    duties_mat.loc['U.S.', 'Europe']  += tariff

    vcosts_h = tcosts.add(pcosts_rev['rm_h'], axis=0).add(pcosts_rev['pc_h'], axis=0) * duties_mat
    vcosts_r = tcosts.add(pcosts_rev['rm_r'], axis=0).add(pcosts_rev['pc_r'], axis=0) * duties_mat

    fc = pcosts_rev[['fc_p','fc_h','fc_r']].values
    vh = (vcosts_h * x_h).values
    vr = (vcosts_r * x_r).values
    total_cost = sum(0.2 * fc[i,j] for i in n_ctry for j in n_lines) + sum(0.8 * fc[i,j] * x_plant[i,j] for i in n_ctry for j in n_lines) + sum(vh[i,j] for i in n_ctry for j in n_ctry) + sum(vr[i,j] for i in n_ctry for j in n_ctry)

    return total_cost

# Calculate excess capacity given decision variables
def calc_excess_cap(dec_plant, dec_h, dec_r):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    excess_cap = (x_plant * caps.values).copy()
    excess_cap[:, 0] -= (np.sum(x_h, axis=1) + np.sum(x_r, axis=1))
    excess_cap[:, 1] -= np.sum(x_h, axis=1)
    excess_cap[:, 2] -= np.sum(x_r, axis=1)
    return excess_cap

# Calculate unmet demand given decision variables
def calc_unmet_demand(dec_h, dec_r):
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    x_h_sum = np.sum(x_h, axis=0)
    x_r_sum = np.sum(x_r, axis=0)
    unmet_demand = (demand.values).copy()
    unmet_demand = np.column_stack((x_h_sum - unmet_demand[:, 0], x_r_sum - unmet_demand[:, 1]))

    return unmet_demand

### Question 1: 

In [12]:
# Sample 100 draws from the exchange rate distribution
exchange_rate_draws = mvn_distribution.rvs(size=100, random_state=42)

# Create a Gurobi model
tariff = 0
based_yr = 2019
# Loop over each draw and optimize the model
for draw in range(100):
    model = Model("MinimizeCost")
    # Extract the i-th draw
    exrate_draw = pd.Series(exchange_rate_draws[draw], index=exrate.index)
    
    # Assign initial value of decision variables
    dec_plant = {(i, j): 1 for i in n_ctry for j in n_lines}
    dec_h = {(i, j): 1 for i in n_ctry for j in n_ctry}
    dec_r = {(i, j): 1 for i in n_ctry for j in n_ctry}

    # Define decision variables
    dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}") for i in n_ctry for j in n_lines}
    dec_h = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
    dec_r = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}
    
    # Excess Capacity constraints
    excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
    for i in n_ctry:
        for j in n_lines:
            model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")

    # Unmet demand constraints
    unmet_demand = calc_unmet_demand(dec_h, dec_r)
    for i in n_ctry:
        for j in range(2):
            model.addConstr(unmet_demand[i, j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")

    # Update the model
    model.update()

    # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
    model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, based_yr, exrate_draw, tariff), GRB.MINIMIZE)

    # Suppress optimization output
    model.Params.OutputFlag = 0

    # Optimize the model
    model.optimize()

    # Extract results to print as a table
    op_plant = pd.DataFrame([[dec_plant[i, j].x for j in n_lines] for i in n_ctry], columns=['Plant', 'H', 'R'], index=caps.index)
    op_h = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns=tcosts.columns, index=tcosts.index)
    op_r = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns=tcosts.columns, index=tcosts.index)

    # Print the results for each draw
    print(f"\nResults for Draw {draw+1}:\n")
    print("HighCal Flow\n")
    print(tabulate(op_h, headers='keys', tablefmt='pretty'))
    print("\nRelax Flow\n")
    print(tabulate(op_r, headers='keys', tablefmt='pretty'))
    print("\nStrategy\n")
    print(tabulate(op_plant, headers='keys', tablefmt='pretty'))
    print(f"\nMinimum Cost: $ {round(model.objVal, 2)} in year {selected_yr} at Tariff {(tariff*100)}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-10

Results for Draw 1:

HighCal Flow

+---------+--------------+-------------------+-------------+-------+--------+--------------------+
|  from   | LatinAmerica |      Europe       | AsiaWoJapan | Japan | Mexico |        U.S.        |
+---------+--------------+-------------------+-------------+-------+--------+--------------------+
| Brazil  |     7.0      |        0.0        |     0.0     |  0.0  |  0.0   |        0.0         |
| Germany |     0.0      |        0.0        |     0.0     |  0.0  |  0.0   |        0.0         |
|  India  |     0.0      | 2.999999999998721 |     5.0     |  7.0  |  0.0   |        0.0         |
|  Japan  |     0.0      |        0.0        |     0.0     |  0.0  |  0.0   |        0.0         |
| Mexico  |     0.0      | 8.000000000001279 |     0.0     |  0.0  |  3.0   |        0.0         |
|  U.S.   |     0.0      | 4.000000000000001 |     0.0     |  0.0  |  0.0   | 17.


Results for Draw 7:

HighCal Flow

+---------+--------------+--------------------+-------------+-------+--------+--------------------+
|  from   | LatinAmerica |       Europe       | AsiaWoJapan | Japan | Mexico |        U.S.        |
+---------+--------------+--------------------+-------------+-------+--------+--------------------+
| Brazil  |     7.0      |        0.0         |     0.0     |  0.0  |  0.0   |        0.0         |
| Germany |     0.0      |        0.0         |     0.0     |  0.0  |  0.0   |        0.0         |
|  India  |     0.0      |        3.0         |     5.0     |  7.0  |  0.0   |        0.0         |
|  Japan  |     0.0      |        0.0         |     0.0     |  0.0  |  0.0   |        0.0         |
| Mexico  |     0.0      | 11.999999999999998 |     0.0     |  0.0  |  3.0   | 12.000000000000002 |
|  U.S.   |     0.0      |        0.0         |     0.0     |  0.0  |  0.0   | 5.999999999999998  |
+---------+--------------+--------------------+-------------+---


Results for Draw 14:

HighCal Flow

+---------+--------------+--------------------+-------------+-------+--------+--------------------+
|  from   | LatinAmerica |       Europe       | AsiaWoJapan | Japan | Mexico |        U.S.        |
+---------+--------------+--------------------+-------------+-------+--------+--------------------+
| Brazil  |     7.0      |        0.0         |     0.0     |  0.0  |  0.0   |        0.0         |
| Germany |     0.0      |        0.0         |     0.0     |  0.0  |  0.0   |        0.0         |
|  India  |     0.0      |        3.0         |     5.0     |  7.0  |  0.0   |        0.0         |
|  Japan  |     0.0      |        0.0         |     0.0     |  0.0  |  0.0   |        0.0         |
| Mexico  |     0.0      | 11.999999999999996 |     0.0     |  0.0  |  3.0   | 12.000000000000004 |
|  U.S.   |     0.0      |        0.0         |     0.0     |  0.0  |  0.0   | 5.999999999999997  |
+---------+--------------+--------------------+-------------+--


Results for Draw 21:

HighCal Flow

+---------+--------------+--------+-------------+-------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------+-------------+-------+--------+------+
| Brazil  |     7.0      |  4.0   |     0.0     |  0.0  |  0.0   | 0.0  |
| Germany |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
|  India  |     0.0      |  3.0   |     5.0     |  7.0  |  0.0   | 0.0  |
|  Japan  |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
| Mexico  |     0.0      |  4.0   |     0.0     |  0.0  |  3.0   | 0.0  |
|  U.S.   |     0.0      |  4.0   |     0.0     |  0.0  |  0.0   | 18.0 |
+---------+--------------+--------+-------------+-------+--------+------+

Relax Flow

+---------+--------------+--------+-------------+-------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------+-------------+-------+-----


Results for Draw 26:

HighCal Flow

+---------+--------------+--------+-------------+-------------------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan |       Japan       | Mexico | U.S. |
+---------+--------------+--------+-------------+-------------------+--------+------+
| Brazil  |     7.0      |  0.0   |     0.0     |        0.0        |  0.0   | 0.0  |
| Germany |     0.0      |  0.0   |     0.0     |        0.0        |  0.0   | 0.0  |
|  India  |     0.0      |  0.0   |     5.0     | 6.999999999999999 |  0.0   | 0.0  |
|  Japan  |     0.0      |  0.0   |     0.0     |        0.0        |  0.0   | 0.0  |
| Mexico  |     0.0      |  11.0  |     0.0     |        0.0        |  3.0   | 0.0  |
|  U.S.   |     0.0      |  4.0   |     0.0     |        0.0        |  0.0   | 18.0 |
+---------+--------------+--------+-------------+-------------------+--------+------+

Relax Flow

+---------+--------------+-------------------+-------------+-------+--------+------+
|  fr

...


Results for Draw 69:

HighCal Flow

+---------+--------------+--------------------+-------------+-------+--------+------+
|  from   | LatinAmerica |       Europe       | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------------------+-------------+-------+--------+------+
| Brazil  |     7.0      |        4.0         |     0.0     |  0.0  |  0.0   | 0.0  |
| Germany |     0.0      |        0.0         |     0.0     |  0.0  |  0.0   | 0.0  |
|  India  |     0.0      | 2.9999999999999996 |     5.0     |  7.0  |  0.0   | 0.0  |
|  Japan  |     0.0      |        0.0         |     0.0     |  0.0  |  0.0   | 0.0  |
| Mexico  |     0.0      |        4.0         |     0.0     |  0.0  |  3.0   | 0.0  |
|  U.S.   |     0.0      |        4.0         |     0.0     |  0.0  |  0.0   | 18.0 |
+---------+--------------+--------------------+-------------+-------+--------+------+

Relax Flow

+---------+--------------+--------+-------------+-------+--------+------+
|  from   | Lati


Results for Draw 76:

HighCal Flow

+---------+--------------+--------+-------------+-------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------+-------------+-------+--------+------+
| Brazil  |     7.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
| Germany |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
|  India  |     0.0      |  0.0   |     5.0     |  7.0  |  0.0   | 0.0  |
|  Japan  |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
| Mexico  |     0.0      |  11.0  |     0.0     |  0.0  |  3.0   | 0.0  |
|  U.S.   |     0.0      |  4.0   |     0.0     |  0.0  |  0.0   | 18.0 |
+---------+--------------+--------+-------------+-------+--------+------+

Relax Flow

+---------+--------------+--------+-------------+-------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------+-------------+-------+-----


Results for Draw 83:

HighCal Flow

+---------+--------------+--------------------+-------------+-------+--------+--------------------+
|  from   | LatinAmerica |       Europe       | AsiaWoJapan | Japan | Mexico |        U.S.        |
+---------+--------------+--------------------+-------------+-------+--------+--------------------+
| Brazil  |     7.0      | 1.0000000000000036 |     0.0     |  0.0  |  0.0   |        0.0         |
| Germany |     0.0      |        0.0         |     0.0     |  0.0  |  0.0   |        0.0         |
|  India  |     0.0      | 2.9999999999999996 |     5.0     |  7.0  |  0.0   |        0.0         |
|  Japan  |     0.0      |        0.0         |     0.0     |  0.0  |  0.0   |        0.0         |
| Mexico  |     0.0      |        7.0         |     0.0     |  0.0  |  3.0   |        0.0         |
|  U.S.   |     0.0      | 3.9999999999999964 |     0.0     |  0.0  |  0.0   | 17.999999999999996 |
+---------+--------------+--------------------+-------------+--


Results for Draw 89:

HighCal Flow

+---------+--------------+--------+-------------+-------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------+-------------+-------+--------+------+
| Brazil  |     7.0      |  4.0   |     0.0     |  0.0  |  0.0   | 0.0  |
| Germany |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
|  India  |     0.0      |  0.0   |     5.0     |  7.0  |  0.0   | 0.0  |
|  Japan  |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
| Mexico  |     0.0      |  7.0   |     0.0     |  0.0  |  3.0   | 0.0  |
|  U.S.   |     0.0      |  4.0   |     0.0     |  0.0  |  0.0   | 18.0 |
+---------+--------------+--------+-------------+-------+--------+------+

Relax Flow

+---------+--------------+--------------------+-------------+-------+--------+------+
|  from   | LatinAmerica |       Europe       | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+------------


Results for Draw 95:

HighCal Flow

+---------+--------------+--------+-------------+-------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------+-------------+-------+--------+------+
| Brazil  |     7.0      |  0.0   |     0.0     |  4.0  |  0.0   | 0.0  |
| Germany |     0.0      |  15.0  |     0.0     |  1.0  |  0.0   | 1.0  |
|  India  |     0.0      |  0.0   |     5.0     |  2.0  |  0.0   | 0.0  |
|  Japan  |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
| Mexico  |     0.0      |  0.0   |     0.0     |  0.0  |  3.0   | 17.0 |
|  U.S.   |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
+---------+--------------+--------+-------------+-------+--------+------+

Relax Flow

+---------+--------------+-------------------+-------------+-------+--------+------+
|  from   | LatinAmerica |      Europe       | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------------

### Question 2

Strategy A focuses on maintaining operations in Brazil, India, and Mexico due to their consistent importance across scenarios. This strategy capitalizes on the stability and critical role of these plants, ensuring the network leverages their strategic locations and efficiencies. Strategy B leverages the near-inverse operation rates of Germany and Japan, suggesting a tactical approach to optimize costs based on exchange rate predictions. This strategy indicates a nuanced understanding of the economic impacts on operational costs, allowing for dynamic adjustments between these two plants to maintain cost efficiency. Strategy C emphasizes operational flexibility with a focus on dynamically adjusting the operations of Germany and Japan, while keeping Brazil, India, and Mexico open. This strategy combines the reliability of certain plants with the flexibility of others, aiming for a balanced approach that can adapt to varying economic conditions. Each strategy is crafted based on the operational data from the 100 draws, focusing on leveraging consistent patterns and correlations between plant operations and exchange rates to formulate a robust network strategy.

In [ ]:
"""
Implementation Plan:
"""

# Step 1: Sample 100 Draws from the Exchange Rate Distribution
# We've already handled this

# Step 2: Determine Optimal Network Configuration for Each Draw
# Modify the optimization model to iterate over each draw from the exchange rate samples
# and optimize the network configuration based on the sampled exchange rates.

# Step 3: Analyze the Results to Propose 3 Candidate Network Strategies
# After running the optimization for each draw, record which plants/lines are open for each scenario.
# Analyze these records to identify patterns and propose 3 candidate strategies.

# Step 4: Evaluate Proposed Strategies
# For each of the 3 strategies, "lock" the plant/line decisions as per the strategy,
# and evaluate the performance by simulating 100 new draws of exchange rates.
# For this, we'll modify the optimization constraints to fix the open/close status of plants/lines.

# Step 5: Make a Recommendation
# Based on the evaluation, calculate the expected cost and cost standard deviation for each strategy.
# Recommend the strategy with the best balance between low average cost and low cost variability.

"""
Pseudo-Code for Key Steps:
"""

#'mvn_distribution' is the multivariate normal distribution object for exchange rates:

# Step 1 is already implemented above.

# For Step 2, we would modify the loop where we optimize the model for each draw like so:
"""
for draw in exchange_rate_draws:
    # Set the exchange rates in the model based on 'draw'
    # Optimize the network configuration
    # Record the open/close status of each plant/line
"""

# For Step 3, after collecting data:
"""
# Analyze the frequency of open/close status for each plant/line
# Identify patterns and propose strategies based on these frequencies
"""

# For Step 4, to evaluate each strategy:
"""
for strategy in proposed_strategies:
    for draw in new_exchange_rate_draws:  # New set of 100 draws
        # Fix the plant/line decisions based on 'strategy'
        # Calculate the cost without optimizing plant/line decisions
    # Compute the average cost and standard deviation for 'strategy'
"""

# For Step 5, based on the evaluations:
"""
# Compare the average costs and standard deviations
# Recommend the strategy with the optimal cost-performance balance
"""

In [ ]:
# Function to optimize network configuration for a given exchange rate draw
def optimize_network(draw, caps, pcosts, tcosts, duties, demand):
    # Initialize the model
    model = Model("network_optimization")
    model.setParam('OutputFlag', 0)
    
    plants = caps.index.tolist()
    markets = demand.index.tolist()
    products = ['Highcal', 'Relax']

    # Decision Variables
    # Plant open/shut
    open_plant = model.addVars(plants, vtype=GRB.BINARY, name="open_plant")
    # Product lines on/off
    line_on = model.addVars(plants, products, vtype=GRB.BINARY, name="operate_line")
    # Quantities shipped
    ship_qty = model.addVars(plants, markets, products, vtype=GRB.CONTINUOUS, name="ship_qty")

    # Constraints
    # Capacity constraints
    for plant in plants:
        model.addConstr(sum(ship_qty[plant, market, product] for market in markets for product in products) <= caps.loc[plant, 'cap'] * open_plant[plant], name=f"capacity_{plant}")

    # Demand constraints
    for market in markets:
        for product in products:
            model.addConstr(sum(ship_qty[plant, market, product] for plant in plants) == demand.loc[market, f'd_{product[0].lower()}'], name=f"demand_{market}_{product}")

    # Objective Function: Minimize total cost
    total_cost = quicksum(
        # Fixed and variable production costs
        open_plant[plant] * pcosts.loc[plant, 'fc_p'] +
        line_on[plant, product] * pcosts.loc[plant, f'fc_{product[0].lower()}'] +
        # Shipping costs
        ship_qty[plant, market, product] * (tcosts.loc[plant, market] + duties.loc[market, 'duty'] + pcosts.loc[plant, f'pc_{product[0].lower()}'])
        for plant in plants for market in markets for product in products
    )
    
    model.setObjective(total_cost, GRB.MINIMIZE)
    model.optimize()

    # Extract solution
    if model.status == GRB.OPTIMAL:
        solution = {
            'open_plants': {plant: open_plant[plant].x for plant in plants},
            'operate_lines': {(plant, product): line_on[plant, product].x for plant in plants for product in products},
            'ship_qty': {(plant, market, product): ship_qty[plant, market, product].x for plant in plants for market in markets for product in products}
        }
        return solution
    else:
        return None

In [ ]:
# Placeholder for optimization results
optimal_configs = []

# Loop over each exchange rate draw to optimize network configuration
optimal_configs = [optimize_network(draw, caps, pcosts, tcosts, duties, demand) for draw in exchange_rate_draws]

In [ ]:
for config in optimal_configs:
    open_plants_df = pd.DataFrame.from_dict(config['open_plants'], orient='index', columns=['Status'])
    open_plants_df['Status'] = open_plants_df['Status'].apply(lambda x: 'Open' if x > 0 else 'Closed')
    
    operate_lines_df = pd.DataFrame.from_dict(config['operate_lines'], orient='index', columns=['Status'])
    operate_lines_df['Status'] = operate_lines_df['Status'].apply(lambda x: 'Operate' if x > 0 else 'Do not operate')
    
    ship_qty_df = pd.DataFrame.from_dict(config['ship_qty'], orient='index', columns=['Quantity'])
    
    print("Open Plants:")
    print(tabulate(open_plants_df, headers='keys', tablefmt='pretty'))
    print("\nOperate Lines:")
    print(tabulate(operate_lines_df, headers='keys', tablefmt='pretty'))
    print("\nShipment Quantities:")
    print(tabulate(ship_qty_df, headers='keys', tablefmt='pretty'))
    print("\n" + "="*50 + "\n")

In [ ]:
# Step 2: Optimizing for each draw
for draw in exchange_rate_draws:
    config = optimize_network(draw, caps, pcosts, tcosts, duties, demand, exchange_rate_draws)
    optimal_configs.append(config)

# Analyze the optimal configurations to propose 3 candidate network strategies
# You would need to implement logic to analyze 'optimal_configs' and derive strategies

# Step 3: Evaluate proposed strategies
# Placeholder for evaluating strategies
# Implement your logic to evaluate each proposed strategy based on new exchange rate samples

# Final Step: Make a recommendation based on evaluation
# Determine which of the proposed strategies performs best based on your evaluation criteria

# Note: This script provides a high-level structure for solving the given problem.
# You need to fill in the detailed optimization model, analysis, and evaluation logic based on your specific case study requirements.

### Question 3

We were unable to fully execute our strategies through codes. Therefore, here is a description of our initial approach and incomplete code:

Loop through each exchange rate sample, apply each strategy, and calculate the operational costs. Collect these costs into arrays or lists, then calculate the mean and standard deviation for each strategy.  Lower cost and lower standard deviation is what we want.  Lower standard deviation implies more predictability and less risk in cost variation. 

In [ ]:
import numpy as np


costs_strategy_a = []
costs_strategy_b = []
costs_strategy_c = []

for index, row in exchange_rates_df.iterrows():
   
    exchange_rate = row['ExchangeRate']

    # Apply Strategy A
    cost_a = calculate_cost(strategy='A', exchange_rate=exchange_rate)
    costs_strategy_a.append(cost_a)
    
    # Apply Strategy B
    cost_b = calculate_cost(strategy='B', exchange_rate=exchange_rate)
    costs_strategy_b.append(cost_b)
    
    # Apply Strategy C
    cost_c = calculate_cost(strategy='C', exchange_rate=exchange_rate)
    costs_strategy_c.append(cost_c)

# Calculate mean and standard deviation for each strategy
mean_a, std_a = np.mean(costs_strategy_a), np.std(costs_strategy_a)
mean_b, std_b = np.mean(costs_strategy_b), np.std(costs_strategy_b)
mean_c, std_c = np.mean(costs_strategy_c), np.std(costs_strategy_c)

# Output the results for analysis
print(f"Strategy A: Mean Cost = {mean_a}, Std Dev = {std_a}")
print(f"Strategy B: Mean Cost = {mean_b}, Std Dev = {std_b}")
print(f"Strategy C: Mean Cost = {mean_c}, Std Dev = {std_c}")